<a href="https://colab.research.google.com/github/ctrivino1/ML_class_final/blob/main/Chris_Trivino_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [45]:
!pip install pyforest
!pip install git+https://github.com/keras-team/keras-preprocessing.git
import pyforest
import shutil, os
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory

  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-6osn5wz8
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-6osn5wz8


In [46]:
!git clone https://github.com/ctrivino1/ML_class_final.git

fatal: destination path 'ML_class_final' already exists and is not an empty directory.


In [47]:
training_dir = "/content/ML_class_final/train_imgs_1"
test_dir = "/content/ML_class_final/test_imgs_1"
df=pd.read_csv("/content/ML_class_final/train_1.csv")
df.label = df.label.astype(str)

<IPython.core.display.Javascript object>

In [59]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=.2,zoom_range=.2,horizontal_flip=True,width_shift_range=0.05,height_shift_range=0.05,validation_split=.2,)


train_generator = train_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(64,64), 
                                         batch_size=32,
                                         validation_split=.2,
                                         subset="training",
                                         seed=69,

                                         )






Found 13694 validated image filenames belonging to 5 classes.


In [60]:

valid_generator = train_datagen.flow_from_dataframe(dataframe=df, 
                                         directory=training_dir, 
                                         x_col="image_id", 
                                         y_col="label", 
                                         class_mode="categorical", 
                                         target_size=(64,64), 
                                         batch_size=32,
                                         subset="validation",
                                         seed=69)

Found 3423 validated image filenames belonging to 5 classes.


In [67]:

base_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(64,64,3), pooling="max"
)

# Freeze the base model
base_model.trainable = False

# Adds ResNet50 into a model with a output layers with 5 classifaction nodes
model = keras.Sequential([base_model,
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(5,activation="softmax") # number of perceptrons
])
model.compile(loss="categorical_crossentropy",
optimizer="adam", metrics=['acc'])
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit(train_generator,
          validation_data = valid_generator,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_steps = STEP_SIZE_VALID,
          epochs=5)




Epoch 1/5
427/427 [==============================] - 239s 552ms/step - loss: 1.1664 - acc: 0.6134 - val_loss: 1.1147 - val_acc: 0.6203
Epoch 2/5
427/427 [==============================] - 234s 548ms/step - loss: 1.1416 - acc: 0.6159 - val_loss: 1.1133 - val_acc: 0.6221
Epoch 3/5
427/427 [==============================] - 234s 547ms/step - loss: 1.1360 - acc: 0.6170 - val_loss: 1.1114 - val_acc: 0.6247
Epoch 4/5
427/427 [==============================] - 233s 545ms/step - loss: 1.1267 - acc: 0.6172 - val_loss: 1.1220 - val_acc: 0.6223
Epoch 5/5
427/427 [==============================] - 234s 547ms/step - loss: 1.1201 - acc: 0.6184 - val_loss: 1.1211 - val_acc: 0.6200
